# Taller de SQL con Olist (Redshift)

Este notebook contiene ejercicios de SQL avanzado usando el dataset **Olist** cargado en un clúster de **Amazon Redshift**.


> - Todos los ejercicios deben resolverse escribiendo consultas SQL en Redshift (usando las funciones de ayuda definidas abajo).


## 1. Conexión a Amazon Redshift

La conexión usa **variables de entorno**, NO credenciales en claro en el código.

Variables de entorno esperadas:

- `REDSHIFT_HOST`  – endpoint de Redshift (sin `https://`, solo el host)
- `REDSHIFT_PORT`  – puerto (opcional, por defecto 5439)
- `REDSHIFT_DB`    – nombre de la base de datos (por defecto `dev` si no se define)
- `REDSHIFT_USER`  – usuario
- `REDSHIFT_PASSWORD` – contraseña


In [ ]:
# Si necesitas instalar el conector de Redshift en tu entorno local, descomenta y ejecuta:
# !pip install redshift-connector pandas

import os
import redshift_connector
import pandas as pd

def get_redshift_config_from_env():
    """Carga la configuración de Redshift desde variables de entorno."""
    missing = []
    host = os.environ.get("REDSHIFT_HOST")
    user = os.environ.get("REDSHIFT_USER")
    password = os.environ.get("REDSHIFT_PASSWORD")
    db = os.environ.get("REDSHIFT_DB", "dev")
    port = int(os.environ.get("REDSHIFT_PORT", "5439"))

    if host is None:
        missing.append("REDSHIFT_HOST")
    if user is None:
        missing.append("REDSHIFT_USER")
    if password is None:
        missing.append("REDSHIFT_PASSWORD")

    if missing:
        raise RuntimeError(
            "Faltan variables de entorno para conectar a Redshift: "
            + ", ".join(missing)
        )

    return {
        "host": host,
        "port": port,
        "database": db,
        "user": user,
        "password": password,
    }

REDSHIFT_CONFIG = get_redshift_config_from_env()

conn = redshift_connector.connect(
    host=REDSHIFT_CONFIG["host"],
    port=REDSHIFT_CONFIG["port"],
    database=REDSHIFT_CONFIG["database"],
    user=REDSHIFT_CONFIG["user"],
    password=REDSHIFT_CONFIG["password"]
)

def execute(sql: str):
    """Ejecuta un comando SQL (DDL/DML) sin devolver resultados (CREATE, INSERT, etc.)."""
    with conn.cursor() as cur:
        cur.execute(sql)
    conn.commit()

def query(sql: str) -> pd.DataFrame:
    """Ejecuta un SELECT y devuelve un DataFrame con los resultados."""
    with conn.cursor() as cur:
        cur.execute(sql)
        cols = [d[0] for d in cur.description]
        rows = cur.fetchall()
    return pd.DataFrame(rows, columns=cols)

# Prueba rápida de conexión (modifica la query si lo deseas)
df_test = query("SELECT 1 AS ok;")
df_test

## 2. Esquema y tablas de Olist


- `olist.orders`
- `olist.order_items`
- `olist.order_payments`
- `olist.customers`
- `olist.products`
- `olist.order_reviews`




### 2.1. (Opcional) Explorar esquema `olist`

In [ ]:
# Usa esta celda para inspeccionar las tablas y columnas disponibles en tu esquema Olist.
query("""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'olist'
ORDER BY table_name, ordinal_position;
""")

---
## 3. Ejercicios

A continuación encontrarás una serie de ejercicios.  
Cada ejercicio tiene una celda de código vacía para que escribas la consulta SQL correspondiente.

> Sugerencia: construye tus consultas usando `query(""" ... """)` para ver los resultados como DataFrame.


### Ejercicio 2 – Ticket promedio por año y estado del pedido

Usando `olist.f_order_items`:

1. Calcula el **importe total** de cada pedido (suma de `total_item_value` por `order_id`).  
2. Obtén el **ticket promedio** por combinación de:
   - Año de compra (derivado de `order_date` o `order_purchase_timestamp`).
   - `order_status`.

Ordena el resultado por año y estado.


In [ ]:
# Escribe aquí tu consulta para el ticket promedio por año y estado


### Ejercicio 3 – Top 10 ciudades por facturación y % sobre el total

Calcula la facturación total (`SUM(total_item_value)`) por ciudad (`customer_city`) y:

1. Obtén el **Top 10** de ciudades por facturación.  
2. Para cada una, calcula el **porcentaje** que representa respecto al total facturado en todo el dataset.

Ordena de mayor a menor facturación.


In [ ]:
# Escribe aquí tu consulta para el Top 10 de ciudades por facturación


### Ejercicio 4 – Top 3 productos por categoría (funciones ventana)

Para cada categoría de producto (`product_category_name`):

1. Calcula la facturación total por producto.  
2. Obtén el **Top 3** de productos por facturación dentro de cada categoría usando funciones ventana (`RANK`, `DENSE_RANK` o similar).

Ordena el resultado por categoría y ranking.


In [ ]:
# Escribe aquí tu consulta con funciones ventana para el Top 3 por categoría


### Ejercicio 5 – Ranking de vendedores y participación acumulada

Construye un ranking de vendedores por **ingresos anuales**, donde:

1. Para cada año, calcules el revenue total por `seller_id`.  
2. Asignes un **ranking** por año (1 = mayor revenue).  
3. Calcules la **participación anual** de cada vendedor (% sobre el total de ese año).  
4. Calcules la **participación acumulada** ordenada de mayor a menor (para ver, por ejemplo, qué porcentaje del revenue acumulan los 20 vendedores más grandes).

Limita la salida a los 20 primeros vendedores por año.


In [ ]:
# Escribe aquí tu consulta de ranking de vendedores con funciones ventana


### Ejercicio 6 – Métricas de entrega y SLA

Usando `olist.orders`:

1. Calcula, en días, el **tiempo de entrega** de cada pedido (`delivery_time_days` = `order_delivered_customer_date` - `order_purchase_timestamp`).  
2. Clasifica cada pedido en:
   - `on_time` si se entregó en o antes de la fecha estimada (`order_estimated_delivery_date`).
   - `late` si se entregó después de la fecha estimada.
   - `not_delivered` si no hay fecha de entrega.

Guarda este resultado en una tabla `olist.f_order_delivery` (o en una CTE si lo prefieres).

Luego, calcula por **estado del cliente (`customer_state`)**:

- El tiempo medio de entrega.
- Percentiles 50, 90 y 95 del tiempo de entrega.
- El porcentaje de pedidos `late`.

Ordena de peor a mejor desempeño (por ejemplo, por mayor tiempo medio de entrega o mayor % de `late`).


In [ ]:
# Escribe aquí tus consultas para crear f_order_delivery y calcular métricas por estado


### Ejercicio 7 – Cohortes por mes de primera compra

Define la **cohorte** de cada cliente como el mes de su primera compra.

1. Para cada `customer_id`, identifica la fecha de su primer pedido.  
2. Define `cohort_month` como el primer día del mes de esa primera compra.  
3. Construye una tabla (o vista) que asigne a cada pedido la cohorte de su cliente.

Luego, arma una tabla que para cada cohorte y cada **mes relativo** (0 = mes de primera compra, 1 = siguiente, etc.) muestre el número de pedidos.

> Pista: puedes calcular el índice de mes relativo usando diferencias de año y mes entre `order_month` y `cohort_month`.


In [ ]:
# Escribe aquí tus consultas para calcular cohortes y tabla de pedidos por cohorte/mes relativo


### Ejercicio 8 – ROLLUP: revenue por estado y categoría

Usando `olist.f_order_items` y tablas relacionadas:

1. Calcula el revenue total por combinación de:
   - Estado del cliente (`customer_state`).
   - Categoría de producto (`product_category_name`).

2. Usa `ROLLUP(customer_state, product_category_name)` para obtener además:
   - Totales por estado.
   - Total general.

Incluye en la salida alguna forma de distinguir las filas de totales (por ejemplo usando `GROUPING` si tu motor lo soporta).


In [ ]:
# Escribe aquí tu consulta con ROLLUP


### Ejercicio 9 – GROUPING SETS: múltiples vistas en una sola query

En una única consulta, calcula el revenue total:

- Por estado del cliente (`customer_state`).
- Por categoría de producto (`product_category_name`).
- Por método de pago (`payment_type`).
- Total general.

Usa `GROUPING SETS` para evitar escribir varias consultas separadas.


In [ ]:
# Escribe aquí tu consulta con GROUPING SETS


### Ejercicio 10 – Clientes recurrentes vs no recurrentes

Define:

- **Cliente recurrente:** cliente con 3 o más pedidos.  
- **Cliente no recurrente:** cliente con 1 o 2 pedidos.

Calcula, al menos:

1. Número de clientes recurrentes y no recurrentes.  
2. Revenue total y ticket promedio de cada grupo.  
3. Distribución de `customer_state` en cada grupo (porcentaje por estado).

Comenta brevemente tus hallazgos en una celda de texto (Markdown).


In [ ]:
# Escribe aquí tus consultas sobre clientes recurrentes vs no recurrentes


### Ejercicio 11 – Reviews, tiempos de entrega y satisfacción

Usando `olist.order_reviews`, `olist.orders` (y si lo deseas `olist.f_order_delivery`):

1. Calcula la **relación** entre `review_score` y el tiempo de entrega (`delivery_time_days`).
   - Por ejemplo, promedio de `delivery_time_days` por `review_score`.
2. Calcula la distribución de `review_score` para pedidos `on_time` vs `late`.  
3. Identifica las 5 categorías de producto con peor combinación de:
   - Bajo `review_score` promedio.
   - Alto % de entregas tardías.

No es necesario que encuentres “la mejor” métrica; el objetivo es escribir consultas que mezclen varias tablas y agregaciones.


In [ ]:
# Escribe aquí tus consultas combinando reviews y tiempos de entrega


### Ejercicio 12 – (Opcional) CTE recursiva

Diseña una CTE recursiva que genere una tabla de fechas mes a mes entre el mínimo y el máximo `order_purchase_timestamp` presentes en `olist.orders`.

Usa esa tabla de fechas para:

1. Asegurar que todas las combinaciones cohorte/mes relativo existan (incluso si no hay pedidos).  
2. O construir un calendario de referencia para otros análisis.

> Nota: la sintaxis de CTE recursiva puede variar un poco entre motores (PostgreSQL, Redshift, etc.).


In [ ]:
# Escribe aquí tu CTE recursiva y las consultas que la usen
